<a href="https://colab.research.google.com/github/Sudhir22/Plane-Accident-Severity/blob/master/Model_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q xlrd
!git clone https://github.com/Sudhir22/Plane-Accident-Severity.git

Cloning into 'Plane-Accident-Severity'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (17/17), done.


In [0]:
import pandas as pd
import xgboost as xbg
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
from sklearn.model_selection import KFold


In [0]:
train_data=pd.read_csv('Plane-Accident-Severity/train.csv')

In [0]:
train_data.shape

(10000, 13)

In [7]:
train_data.groupby(['Severity']).count()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
Severity,,,,,,,,,,,
Highly_Fatal_And_Damaging,3049,3049,3049,3049,3049,3049,3049,3049,3049,3049,3049
Minor_Damage_And_Injuries,2527,2527,2527,2527,2527,2527,2527,2527,2527,2527,2527
Significant_Damage_And_Fatalities,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695,1695
Significant_Damage_And_Serious_Injuries,2729,2729,2729,2729,2729,2729,2729,2729,2729,2729,2729


In [0]:
train_data['Severity']=pd.Categorical(train_data['Severity'])
train_data['Severity_Code']=train_data['Severity'].cat.codes

In [9]:
X=train_data.iloc[:,1:11]
X.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352
1,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350
2,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364
3,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728
4,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883


In [17]:
Y=train_data.iloc[:,12]
type(Y)

pandas.core.series.Series

In [0]:
kf=KFold(n_splits=5)

In [0]:
param = {
    'eta': 0.4, 
    'max_depth': 12,  
    'objective': 'multi:softprob',  
    'num_class': 4} 

steps = 1000  # The number of training iterations

In [18]:
for x,y in kf.split(X):
  print(x,y)
  X_train,X_test = X.iloc[x,:],X.iloc[y,:]
  Y_train,Y_test = Y[x],Y[y]
  D_train = xbg.DMatrix(X_train, label=Y_train)
  D_test = xbg.DMatrix(X_test, label=Y_test)
  model = xbg.train(param, D_train, steps)
  preds = model.predict(D_test)
  best_preds = np.asarray([np.argmax(line) for line in preds])

  print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
  print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
  print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))
  print("F1-score = {}".format(f1_score(Y_test,best_preds,average='weighted')))
  print("********************************************************************")


[2000 2001 2002 ... 9997 9998 9999] [   0    1    2 ... 1997 1998 1999]


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Precision = 0.9549292619024932
Recall = 0.9558192421725733
Accuracy = 0.9555
F1-score = 0.9554981895365131
********************************************************************
[   0    1    2 ... 9997 9998 9999] [2000 2001 2002 ... 3997 3998 3999]
Precision = 0.9560284847058002
Recall = 0.9597953493997062
Accuracy = 0.9575
F1-score = 0.9574746125584278
********************************************************************
[   0    1    2 ... 9997 9998 9999] [4000 4001 4002 ... 5997 5998 5999]
Precision = 0.9570049982178503
Recall = 0.9622285922634013
Accuracy = 0.9595
F1-score = 0.9594970701914243
********************************************************************
[   0    1    2 ... 9997 9998 9999] [6000 6001 6002 ... 7997 7998 7999]
Precision = 0.9567276142842812
Recall = 0.9586301134182342
Accuracy = 0.9595
F1-score = 0.9595332964158308
********************************************************************
[   0    1    2 ... 7997 7998 7999] [8000 8001 8002 ... 9997 9998 9999]
Precisio

In [0]:
test_data=pd.read_csv("Plane-Accident-Severity/test.csv")

In [0]:
test_data.shape

(2500, 12)

In [0]:
test_X=test_data.iloc[:,0:10]

In [0]:
test_X.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696


In [0]:
test_X.shape

(2500, 10)

In [0]:
D_test_test=xbg.DMatrix(test_X)
test_preds = model.predict(D_test_test)
best_test_preds = np.asarray([np.argmax(line) for line in test_preds])

In [0]:
test_preds.shape

(2500, 4)

In [0]:
severity_dict={1:'Minor_Damage_And_Injuries',2:'Significant_Damage_And_Fatalities',3:'Significant_Damage_And_Serious_Injuries',0:'Highly_Fatal_And_Damaging'}

In [0]:
severity_list=list()
for x in best_test_preds:
  severity_list.append(severity_dict[x])

In [0]:
test_data['Severity']=pd.Series(severity_list)

In [0]:
test_data[['Accident_ID','Severity']].to_csv("Plane-Accident-Severity/submission4.csv",index=False)